In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import super_res
from malaya_speech.train.model import enhancement
import tensorflow as tf
import numpy as np






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.





In [4]:
x = tf.placeholder(tf.float32, (None, 1))
partitioned_x = malaya_speech.tf_featurization.pad_and_partition(x, 16384)
model = super_res.Model(partitioned_x)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Tensor("generator/downsc_conv0/Reshape_2:0", shape=(?, 8192, 128), dtype=float32)
Tensor("generator/downsc_conv1/Reshape_2:0", shape=(?, 4096, 256), dtype=float32)
Tensor("generator/downsc_conv2/Reshape_2:0", shape=(?, 2048, 512), dtype=float32)
Tensor("generator/downsc_conv3/Reshape_2:0", shape=(?, 1024, 512), dtype=float32)
Tensor("generator/bottleneck_conv/Reshape_2:0", shape=(?, 512, 512), dtype=float32)

Tensor("generator/upsc_conv3/concat:0", shape=(?, 1024, 1024), dtype=float32)
Tensor("generator/upsc_conv2/concat:0", shape=(?, 2048, 1024), dtype=float32)
Tensor("generator/upsc_conv1/concat:0", shape=(?, 4096, 512), dtype=float32)
Tensor("generator/upsc_conv0/concat:0", shape=(?, 8192, 256), dtype=float32)
Tensor("generator/lastconv/subpixel/transpose_1:0", shape=(?, 16384, 1), dtype=float32)


In [5]:
model.logits

<tf.Tensor 'generator/add:0' shape=(?, 16384, 1) dtype=float32>

In [6]:
logits = tf.reshape(model.logits, (-1, 1))
logits = logits[:tf.shape(x)[0]]
logits

<tf.Tensor 'strided_slice_4:0' shape=(?, 1) dtype=float32>

In [7]:
snr = enhancement.loss.snr(logits, x)
sdr = enhancement.loss.sdr(logits, x)

In [8]:
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

In [9]:
y, sr = malaya_speech.load('../speech/record/savewav_2020-09-06_21-48-43_174842.wav', sr = 22050)

In [10]:
y_ = sess.run(model.logits, feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(16, 16384, 1)

In [11]:
y_.reshape((-1)).shape

(262144,)

In [12]:
y.shape

(258867,)

In [13]:
import IPython.display as ipd

ipd.Audio(y_.reshape((-1)), rate = 22050)

In [14]:
y_, snr_, sdr_ = sess.run([model.logits, snr, sdr], feed_dict = {x: np.expand_dims(y, -1)})
y_.shape

(16, 16384, 1)

In [15]:
snr_, sdr_

(38.090744, -3.862975e-06)

In [16]:
saver = tf.train.Saver()

In [17]:
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [18]:
!rm -rf test